# Data loading

Training data is distributed in 3 different folders (set a, b and c).
Each file has its raw image (.mhd), lung mask (\_lm.mhd), and fissure mask (\_fm.mhd).

In [1]:
import os, random
import ntpath
import SimpleITK
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from utils import BatchGenerator #custom file for utilities
import callbacks #custom file for callbacks
import pickle
import time

from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, Activation, Deconvolution3D, Cropping3D, UpSampling3D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard


from keras.layers.merge import concatenate
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import RandomOverSampler

C:\Users\denha\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Dataset

In [2]:
# Loading data from pickle:
data = pd.read_pickle("train-data-filelist.pkl")

In [3]:
splitter = StratifiedShuffleSplit(1, test_size=0.1)

In [4]:
for train_index, test_index in splitter.split(data, data['label'].values):
    train_set = data.loc[train_index]
    validation_set = data.loc[test_index]

# Patch Generator

In [5]:
patch_size = (132,132,116) # smallest possible patch size is (108,108,108)
batch_size = 6 # 16 is max due to gpu memory errors

sampler = RandomOverSampler(random_state=0)

train_generator = BatchGenerator(train_set, patch_size, batch_size=batch_size, sampling=sampler.fit_sample)
validation_generator = BatchGenerator(validation_set, patch_size, batch_size=batch_size)

# U-net

In [6]:
# Loss calculation for 3D U-net
def dice_coefficient(y_true, y_pred, smooth=1.):
    bglabel = 0
    cflabel = 2
    iflabel = 4
    importance_factors = [0.001, 0.5, 0.5]
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    penalty_mask = (y_true_f == bglabel) * importance_factors[0] 
    + (y_true_f == cflabel) * importance_factors[1] 
    + (y_true_f == iflabel) * importance_factors[2] 
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)

In [ ]:
def create_network(input_shape, features=32):
    
    # Downward path
    
    levels = list()
    inputs = Input(input_shape)
    # Block 1
    layer1 = Conv3D(features*(2**0), (3,3,3), padding='valid', strides=1)(inputs)
    layer1 = BatchNormalization()(layer1)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(features*(2**0), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Activation('relu')(layer2)
    pool = MaxPooling3D(pool_size=(2,2,2))(layer2)
    levels.append([layer1, layer2, pool])
    # Block 2
    layer1 = Conv3D(features*(2**1), (3,3,3), padding='valid', strides=1)(pool)
    layer1 = BatchNormalization()(layer1)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(features*(2**1), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Activation('relu')(layer2)
    pool = MaxPooling3D(pool_size=(2,2,2))(layer2)
    levels.append([layer1, layer2, pool])
    # Block 3
    layer1 = Conv3D(features*(2**2), (3,3,3), padding='valid', strides=1)(pool)
    layer1 = BatchNormalization()(layer1)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(features*(2**2), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Activation('relu')(layer2)
    pool = MaxPooling3D(pool_size=(2,2,2))(layer2)
    levels.append([layer1, layer2, pool])
    # Block 4
    layer1 = Conv3D(features*(2**3), (3,3,3), padding='valid', strides=1)(pool)
    layer1 = BatchNormalization()(layer1)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(features*(2**3), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Activation('relu')(layer2)
    levels.append([layer1, layer2])

    # Block 5
    layer0 = UpSampling3D(size=2)(layer2)
    #layer0 = Conv3D(32*(2**2), (2,2,2))(layer0)
    crop = levels[2][1]
    size = (crop._keras_shape[-4] - layer0._keras_shape[-4],
            crop._keras_shape[-3] - layer0._keras_shape[-3], 
            crop._keras_shape[-2] - layer0._keras_shape[-2])
    size = ((int(np.floor(size[0]/2)),int(np.ceil(size[0]/2))),
            (int(np.floor(size[1]/2)),int(np.ceil(size[1]/2))),
            (int(np.floor(size[2]/2)),int(np.ceil(size[2]/2))))
    crop = Cropping3D(cropping=size)(crop)
    concatenate([layer0, crop],axis=4)
    layer1 = Conv3D(features*(2**2), (3,3,3), padding='valid', strides=1)(layer0)
    layer1 = BatchNormalization()(layer1)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(features*(2**2), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Activation('relu')(layer2)
    
    # Upward path
    
    # Block 6
    layer0 = UpSampling3D(size=2)(layer2)
    #layer0 = Conv3D(32*(2**1), (2,2,2))(layer0)
    crop = levels[1][1]
    size = (crop._keras_shape[-4] - layer0._keras_shape[-4],
            crop._keras_shape[-3] - layer0._keras_shape[-3], 
            crop._keras_shape[-2] - layer0._keras_shape[-2])
    size = ((int(np.floor(size[0]/2)),int(np.ceil(size[0]/2))),
            (int(np.floor(size[1]/2)),int(np.ceil(size[1]/2))),
            (int(np.floor(size[2]/2)),int(np.ceil(size[2]/2))))
    crop = Cropping3D(cropping=size)(crop)
    concatenate([layer0, crop],axis=4)
    layer1 = Conv3D(features*(2**1), (3,3,3), padding='valid', strides=1)(layer0)
    layer1 = BatchNormalization()(layer1)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(features*(2**1), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Activation('relu')(layer2)
    # Block 7
    layer0 = UpSampling3D(size=2)(layer2)
    #layer0 = Conv3D(32*(2**0), (2,2,2))(layer0)
    crop = levels[0][1]
    size = (crop._keras_shape[-4] - layer0._keras_shape[-4],
            crop._keras_shape[-3] - layer0._keras_shape[-3], 
            crop._keras_shape[-2] - layer0._keras_shape[-2])
    size = ((int(np.floor(size[0]/2)),int(np.ceil(size[0]/2))),
            (int(np.floor(size[1]/2)),int(np.ceil(size[1]/2))),
            (int(np.floor(size[2]/2)),int(np.ceil(size[2]/2))))
    crop = Cropping3D(cropping=size)(crop)
    concatenate([layer0, crop],axis=4)
    layer1 = Conv3D(features*(2**0), (3,3,3), padding='valid', strides=1)(layer0)
    layer1 = BatchNormalization()(layer1)
    layer1 = Activation('relu')(layer1)
    layer2 = Conv3D(features*(2**0), (3,3,3), padding='valid', strides=1)(layer1)
    layer2 = BatchNormalization()(layer2)
    layer2 = Activation('relu')(layer2)

    final = Conv3D(1, (1, 1, 1))(layer2) # 1 output channel due to segmentation image being grayscale like input image patch
    final = Activation('softmax')(final)
    model = Model(inputs=inputs, outputs=final)

    model.compile(optimizer=Adam(lr=0.0001), loss=dice_coefficient_loss, metrics=[dice_coefficient])
    model.summary()
    return model

In [ ]:
model = create_network(input_shape=[*patch_size,1],features=8) 
# nr of feature maps are bottleneck for gpu memory; 8 seems to be max; down from 5 mil parameters to less than 400,000

#logger = Logger(data, patch_size, stride=88) #on training data instead of validation data
timeNow = time.strftime("%e%m-%H%M%S")
#tensorboard = TensorBoard(log_dir='./logs/'+timeNow, batch_size=batch_size, histogram_freq=1, embeddings_freq=0, write_images=True)
modelcheck = ModelCheckpoint("weights-"+str(timeNow)+".hdf5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=90,
                    epochs=10,
                    validation_steps=10,
                    callbacks=[modelcheck])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 132, 132, 116, 1)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 130, 130, 114, 8)  224       
_________________________________________________________________
batch_normalization_1 (Batch (None, 130, 130, 114, 8)  32        
_________________________________________________________________
activation_1 (Activation)    (None, 130, 130, 114, 8)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 128, 128, 112, 8)  1736      
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 112, 8)  32        
_________________________________________________________________
activation_2 (Activation)    (None, 128, 128, 112, 8)  0         
__________

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\denha\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\denha\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\denha\Anaconda3\lib\site-packages\keras\utils\data_utils.py", line 563, in _run
    self.sequence.on_epoch_end()
  File "D:\Documents\GitHub\ISMI-Fissure-Detection\utils.py", line 42, in on_epoch_end
    self.samples, _ = self.sampling(self.indices, self.labels)
  File "C:\Users\denha\Anaconda3\lib\site-packages\imblearn\base.py", line 88, in fit_sample
    return self.fit(X, y).sample(X, y)
  File "C:\Users\denha\Anaconda3\lib\site-packages\imblearn\base.py", line 157, in fit
    X, y = check_X_y(X, y, accept_sparse=['csr', 'csc'])
  File "C:\Users\denha\Anaconda3\lib\site-packages\sklearn\utils\validation.py", line 573, in check_X_y
    ensure_min_features, warn_on_dtype, estimator)
  File "C:\U